<a href="https://colab.research.google.com/github/Sarinezs/Software-Engineering-Project/blob/main/License_Plate_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics easyocr roboflow
!pip install -q streamlit
!npm install -g localtunnel
!npm audit fix --force
from google.colab import drive
drive.mount('/content/drive')

import easyocr
import cv2
import matplotlib.pyplot as plt
from roboflow import Roboflow
from PIL import Image, ImageFont, ImageDraw
import numpy as np
from ultralytics import YOLO


rf = Roboflow(api_key="Jru6dEwbPFxvN1sf5wVF")
project = rf.workspace("roboflow-universe-projects").project("license-plate-recognition-rxg4e")
version = project.version(4)
dataset = version.download("yolov8")

In [ ]:

model = YOLO('yolov8n.pt')

path = '/content/License-Plate-Recognition-4/data.yaml'
results = model.train(data=path,
                      epochs=50,
                      imgsz=640,
                      batch=32,
                      patience=3,
                      optimizer='Adam',
                      lr0=0.001,
                      # project='/content/drive/MyDrive/ML/Machine_Learning_Project',
                      # name='trained_model'
                      )



In [ ]:

model = YOLO('yolov8n.pt')
path = '/content/License-Plate-Recognition-4/data.yaml'

model.tune(data=path,
           epochs=3,
           patience=5,
           optimizer='Adam',
           lr0=0.0001,
           batch=32,
           iterations=3,
           project='/content/drive/MyDrive/ML/Machine_Learning_Project',
           name='tuned_param')




In [ ]:
load_model = YOLO('/content/drive/MyDrive/ML/Machine_Learning_Project/trained_model/weights/best.pt')



In [ ]:
img_path='/content/License-Plate-Recognition-4/test/images/CarLongPlateGen2577_jpg.rf.e24e0e4356f004720e8a339b75d66b94.jpg'

result = load_model.predict(source=img_path,
                            # save=True,
                            imgsz=640,
                            conf=0.5,
                            show_labels=True,
                            show_conf=True,
                            show=True,
                            # save_crop=True,
                            )

reader = easyocr.Reader(['th','en']) # English reader
licenseplate_coor = result[0].boxes.xyxy.cpu().numpy()

if len(licenseplate_coor) == 0:
  print('No license plate detected')
else:
  img=cv2.imread(img_path)

  for i in range (len(licenseplate_coor)):
    lp = licenseplate_coor[i]

    #ตำแหน่งทะเบียน
    start_x = int(lp[0])
    start_y = int(lp[1])
    end_x = int(lp[2])
    end_y = int(lp[3])

    license_plate = img[start_y:end_y, start_x:end_x, :].copy()

    cv2.rectangle(img, (start_x,start_y), (end_x,end_y), (0, 255, 0),2)

    image_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

    font_path = '/content/drive/MyDrive/ML/Machine_Learning_Project/Font/Sarabun-SemiBoldItalic.ttf'
    font = ImageFont.truetype(font_path, 40)

    draw = ImageDraw.Draw(image_pil)

    text_array = ''
    lp_text = reader.readtext(license_plate)
    for i in lp_text:
      bbox, text, score = i
      text_array = text_array + ' ' + text
      print(text,score)

    draw.text((start_x, start_y - 40), text_array, font=font, fill=(0, 0, 255))
    # plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.imshow(cv2.cvtColor(np.array(image_pil), cv2.COLOR_BGR2RGB))

In [ ]:
path = '/content/License-Plate-Recognition-4/data.yaml'
metrics = load_model.val(data=path, imgsz=640, project='/content/drive/MyDrive/ML/Machine_Learning_Project', name='mAP')

In [ ]:
print("Mean Average Precision (mAP) IoU(0.50-0.95) = ",round(metrics.box.map,2))
print("Mean Average Precision (mAP) IoU(0.50)      = ",round(metrics.box.map50,2))
print("Mean Average Precision (mAP)IoU(0.75)       = ",round(metrics.box.map75,2))

In [ ]:
%%writefile app.py

import easyocr
import cv2
import matplotlib.pyplot as plt
import numpy as np
from roboflow import Roboflow
from PIL import Image, ImageFont, ImageDraw
from ultralytics import YOLO
import streamlit as st

st.set_page_config(page_title="License Plate Analysis App")

st.title("License Plate Analysis App")

Conf_threshold = st.sidebar.slider('confidence threshold',
                                   min_value=0.1,
                                   max_value=1.0,
                                   value=0.5,
                                   step=0.01)

load_model = YOLO('/content/drive/MyDrive/ML/Machine_Learning_Project/trained_model/weights/best.pt')

reader = easyocr.Reader(['th','en'])

uploaded_file = st.file_uploader("Choose an image...", type="jpg")


if uploaded_file is not None:

  file_bytes = np.frombuffer(uploaded_file.read(), np.uint8)

  img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)

  result = load_model.predict(source=img,
                              imgsz=640,
                              conf=Conf_threshold,
                              )

  licenseplate_coor = result[0].boxes.xyxy.cpu().numpy()

  if len(licenseplate_coor) == 0:
    st.subheader("No license plate detected")
  else:
    text_array = []

    for i in range (len(licenseplate_coor)):
      lp = licenseplate_coor[i]

      #ตำแหน่งทะเบียน
      start_x = int(lp[0])
      start_y = int(lp[1])
      end_x = int(lp[2])
      end_y = int(lp[3])

      license_plate = img[start_y:end_y, start_x:end_x, :].copy()

      cv2.rectangle(img, (start_x,start_y), (end_x,end_y), (0, 0, 255),2)

      texts = ''
      lp_text = reader.readtext(license_plate)
      for i in lp_text:
        bbox, text, score = i
        texts = texts + ' ' + text

      text_array.append(texts)

    plot_img = result[0].plot()

    plot_img_rgb = cv2.cvtColor(plot_img, cv2.COLOR_BGR2RGB)

    plot_img_pil = Image.fromarray(plot_img_rgb)

    st.image(plot_img_pil, caption="YOLO Prediction", use_container_width=True)
    for text in text_array:
      st.subheader(text)


In [ ]:
!streamlit run /content/app.py & npx localtunnel --port 8501